<a href="https://colab.research.google.com/github/Freaker99/car-detection/blob/main/car_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.flush_and_unmount()
drive.mount('/content/drive')
root = '/content/drive/My Drive/car-detection/'

Mounted at /content/drive


In [ ]:
import shutil


# Ścieżka do folderu źródłowego
src_folder = '/content/drive/My Drive/car-detection/images/train/train_1'

# Ścieżka do folderu docelowego
dst_folder = '/content/drive/My Drive/car-detection/images/train/train100k'

# Przenieś pliki z folderu źródłowego do folderu docelowego
file_list = os.listdir(src_folder)
for file_name in file_list:
    if file_name.endswith('.png'):
        src_path = os.path.join(src_folder, file_name)
        dst_path = os.path.join(dst_folder, file_name)
        shutil.move(src_path, dst_path)
        print(f"Przeniesiono plik: {file_name}")

OSError: ignored

In [ ]:
#!pip install opencv-python
#!pip install torch torchvision
#!pip install numpy
#!pip install ultralytics

In [ ]:
#!git clone https://github.com/ultralytics/yolov5
#%cd yolov5
#!pip install -r requirements.txt
#!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt

In [ ]:
!pip install yolov5
!pip install --upgrade yolov5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importowanie niezbędnych bibliotek
import os
import torch
import json
from torchvision.datasets import VisionDataset
from torchvision.transforms import transforms
from PIL import Image

# Ścieżki do folderów z obrazami
image_dir_train = '/content/drive/My Drive/car-detection/images/train/'
image_dir_test = '/content/drive/My Drive/car-detection/images/val/'
image_dir_val = '/content/drive/My Drive/car-detection/images/val/'

# Ścieżki do plików etykiet
train_labels_file = '/content/drive/My Drive/car-detection/labels/det_train.json'
test_labels_file = '/content/drive/My Drive/car-detection/labels/det_val.json'
val_labels_file = '/content/drive/My Drive/car-detection/labels/det_val.json'

In [ ]:
import json
from torch.utils.data import random_split

# Klasa datasetu BDD100K
class BDD100KDataset(VisionDataset):
    def __init__(self, labels_file, image_dir, transform=None, target_transform=None):
        super(BDD100KDataset, self).__init__(image_dir, transform=transform, target_transform=target_transform)
        self.labels = self._read_labels(labels_file)
        self.images = [item["name"] for item in self.labels]

    def __getitem__(self, index):
        image_path = os.path.join(self.root, self.images[index])
        image = Image.open(image_path).convert("RGB")
        label = self.labels[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.labels)

    def _read_labels(self, labels_file):
        with open(labels_file, "r") as f:
            labels = json.load(f)
        return labels

# Tworzenie instancji datasetu treningowego i walidacyjnego
train_dataset = BDD100KDataset(train_labels_file, image_dir_train)
test_dataset = BDD100KDataset(val_labels_file, image_dir_test)
val_dataset = BDD100KDataset(val_labels_file, image_dir_val)

# Podział datasetu na treningowy i walidacyjny
train_size = int(1 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Definiowanie DataLoader dla treningu i walidacji
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=0)

In [ ]:
import torch
import torch.optim as optim
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from yolov5.models.yolo import Model
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pin_memory = True

# Funkcja treningowa
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0.0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss

In [ ]:
# Inicjalizacja modelu YOLOv5
model = Model(root + "yolov5/models/yolov5s.yaml")

# Załadowanie wstępnie wytrenowanych wag modelu
state_dict = torch.load(root + "yolov5/yolov5s.pt")
model.load_state_dict(state_dict, strict=False)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

# Ustawienie trybu trenowania modelu
model.train()

# Definicja optymalizatora
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Definicja funkcji straty
criterion = torch.nn.CrossEntropyLoss()

# Parametry trenowania
num_epochs = 10
batch_size = 2
learning_rate = 0.001

# Pętla trenowania
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Trening
    train_loss = train(model, train_loader, criterion, optimizer)
    print(f"Train Loss: {train_loss:.4f}")

    # Ocena na zbiorze walidacyjnym (opcjonalnie)
    # val_loss = evaluate(model, val_loader, criterion)
    # print(f"Val Loss: {val_loss:.4f}")

    print("")

# Zapisanie wytrenowanego modelu
torch.save(model.state_dict(), "trained_model.pt")


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]              
  8                -1  1   1182720  

Epoch 1/10


  0%|          | 0/34932 [00:00<?, ?it/s]


FileNotFoundError: ignored

**Tools**

In [ ]:
# PRINT LABELS

import os
import json

image_dir = "/content/drive/My Drive/car-detection/images/train/"
labels_file = "/content/drive/My Drive/car-detection/labels/bdd100k_labels_images_train.json"  # Ścieżka do pliku z etykietami (dostosuj do swojej struktury danych)

# Wczytanie danych etykiet
with open(labels_file, "r") as f:
    labels = json.load(f)

# Sprawdzenie informacji o nazwach plików w etykietach
for item in labels:
    file_name = item["name"]
    # Tutaj możesz wydrukować nazwy plików lub wykonać inne operacje na danych etykiet

# Porównanie nazw plików obrazów z informacjami w etykietach
file_list = os.listdir(image_dir)
for file_name in file_list:
    if file_name not in [item["name"] for item in labels]:
        print("Brak informacji w etykietach dla pliku:", file_name)

In [ ]:
# PRINT FILES

import os

image_dir = "/content/drive/My Drive/car-detection/images/train/"

# Wyświetlenie listy plików w folderze
file_list = os.listdir(image_dir)
for file_name in file_list:
    print(file_name)

In [28]:
# COUNT FILES

import os

# Ścieżka do folderu źródłowego
src_folder = '/content/drive/My Drive/car-detection/images/train/train100k'

# Pobierz listę plików w folderze źródłowym
file_list = os.listdir(src_folder)

# Liczba plików w folderze
num_files = len(file_list)

# Wyświetl liczbę plików
print(f"Liczba plików w folderze: {num_files}")

Liczba plików w folderze: 70000


In [ ]:
# MOVE FILES

import glob
import shutil

# Ścieżka do folderu źródłowego
src_folder = '/content/drive/My Drive/car-detection/images/train/train_4'

# Ścieżka do folderu docelowego
dst_folder = '/content/drive/My Drive/car-detection/images/train/train100k'

file_list = glob.glob(os.path.join(src_folder, '*.jpg'))
batch_size = 100  # Liczba plików w jednej partycji
num_batches = len(file_list) // batch_size + 1

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch_files = file_list[start_idx:end_idx]

    for file_path in batch_files:
        file_name = os.path.basename(file_path)
        dst_path = os.path.join(dst_folder, file_name)
        shutil.move(file_path, dst_path)
        print(f"Przeniesiono plik: {file_name}")